In [80]:
# Installing some libraries
!pip install requests
!pip install pandas
!pip install numpy

In [ ]:

# STEP 3
# Code for Authenthication
from getpass import getpass
import requests #just for check if You can get results from the api

token = getpass("Enter yout Github Token :")

headers = {
    "Authorization": f"Bearer {token}"
}

def test_connection():
  response = requests.get("https://api.github.com/user", headers=headers)
  if response.status_code == 200:
    user = response.json()
    print(f"Succesful opperation. Authenticated user: {user['login']}")
  else:
    print("Connection failed. Check your token !")

test_connection()



Enter yout Github Token :··········
Succesful opperation. Authenticated user: manu97a


In [82]:
## Code for make API requests

# Search Repositories (public)

def search_public_repositories(query = "machine learning", per_page = 10 , page = 1, created_after=None, min_stars=None):
  if per_page > 100:
    print(f"Error. per_page cannot be greater than 100 (Github Limit)")
    return None,None
  if (page * per_page) > 1000:
    print(f"Error. per_page*page cannot be greater than 1000 results (Github Limit)")
    return None,None
  new_query = query
  if created_after:
    new_query += f"+created:>={created_after}"
  if min_stars:
    new_query += f"+stars:>={min_stars}"
  api_url = f"https://api.github.com/search/repositories?q={new_query}&per_page={per_page}&page={page}"
  res = requests.get(api_url, headers=headers)
  if res.status_code == 200:
    repositories = res.json()
    print(f"Repositories for the topic '{query}' : \n")
    for repo in repositories['items'][:5]:
      print(f"{repo['full_name']}")
      print(f"Description: {repo['description']}\n")
    return repositories, api_url
  else:
    print(f"Error {res.status_code}:{res.json()}")
    return None,api_url


In [83]:
# Code for make API requests
#Commits of a repository

def get_commits(owner="midudev",repo= "aprendiendo-react", per_page= 10, since= None, until=None,page=1):
  if per_page > 100:
    print(f"Error. per_page cannot be greater than 100 (Github Limit)")
    return None,None
  if (page * per_page) > 1000:
    print(f"Error. per_page*page cannot be greater than 1000 results (Github Limit)")
    return None,None
  parameters = {
      "per_page": per_page,
      "page":page
  }
  if since:
    parameters["since"] = since
  if until:
    parameters["until"] = until
  api_url = f"https://api.github.com/repos/{owner}/{repo}/commits"
  res = requests.get(api_url, headers=headers,params=parameters)
  if res.status_code == 200:
    commits = res.json()
    print(f"The last {per_page} commits of {owner}/{repo}")
    for commit in commits:
      print(f"{commit['commit']['author']['date']}- {commit['commit']['author']['name']}")
      print(f"{commit['commit']['message']}\n")
    return commits,api_url

  else:
    print(f"Error {res.status_code}:{res.json()}")
    return None,api_url


In [84]:
## Code for make API requests
# Repo content

def repo_content(owner = "midudev", repo= "aprendiendo-react", path=""):
   api_url = f"https://api.github.com/repos/{owner}/{repo}/contents/{path}"
   res = requests.get(api_url, headers=headers)
   if res.status_code == 200:
    content = res.json()
    print(f"Owner:  {owner} \n Name of the repo: {repo}\n Content: {content} \n")
    if isinstance(content,list):
      download_links = []
      for item in content:
        print(f"- {item['type'].capitalize()}: {item['name']}")
        if item['type'] == "file":
          print(f"Download link: {item['download_url']}")
          download_links.append(item['download_url'])
      return download_links
    elif isinstance(content, dict) and content.get("type") == "file":
      print(f"File: {content['name']}")
      print(f"Download link: {content['download_url']}")
      return content['download_url']
    else:
      print("This is not a recognized file or folder")
   else:
    print(f"Error {res.status_code}: {res.json()}")

In [85]:
## Function to handle rate limit
from datetime import datetime
def rate_limit(headers):
  api_url = "https://api.github.com/rate_limit"
  res = requests.get(api_url, headers=headers)
  if res.status_code == 200:
    data = res.json()
    remaining_limit = data['rate']['remaining']
    reset_time = data['rate']['reset']
    reset_datetime = datetime.utcfromtimestamp(reset_time).strftime('%Y-%m-%d %H:%M:%S')
    print(f"Remaining requests: {remaining_limit}")
    print(f"Reset datetime: {reset_datetime}")
  else:
    print(f"Error {res.status_code}:{res.json()}")


rate_limit(headers)



Remaining requests: 4984
Reset datetime: 2025-06-26 22:49:52


**Examples of using functions**

How to use the functions for search public repositories

# 1. Examples for searching public repositories

Search only with a keyword

In [86]:
repos, url = search_public_repositories("javascript")

Repositories for the topic 'javascript' : 

airbnb/javascript
Description: JavaScript Style Guide

TheAlgorithms/JavaScript
Description: Algorithms and Data Structures implemented in JavaScript for beginners, following best practices.

trekhleb/javascript-algorithms
Description: 📝 Algorithms and data structures implemented in JavaScript with explanations and links to further readings

GitbookIO/javascript
Description: GitBook teaching programming basics with Javascript

wesbos/JavaScript30
Description: 30 Day Vanilla JS Challenge



Search repositories created after a date

In [87]:
repos, url = search_public_repositories("machine learning", created_after="2021-07-12")

Repositories for the topic 'machine learning' : 

greyhatguy007/Machine-Learning-Specialization-Coursera
Description:  Contains Solutions and Notes for the Machine Learning Specialization By Stanford University and Deeplearning.ai - Coursera (2022) by Prof. Andrew NG

rasbt/machine-learning-book
Description: Code Repository for Machine Learning with PyTorch and Scikit-Learn

kaieye/2022-Machine-Learning-Specialization
Description: None

Nyandwi/machine_learning_complete
Description: A comprehensive machine learning repository containing 30+ notebooks on different concepts, algorithms and techniques.

christianversloot/machine-learning-articles
Description: 🧠💬 Articles I wrote about machine learning, archived from MachineCurve.com.



Search repositories with a minimun number of stars

In [88]:
repos, url = search_public_repositories("typescript", min_stars=100)

Repositories for the topic 'typescript' : 

microsoft/TypeScript
Description: TypeScript is a superset of JavaScript that compiles to clean JavaScript output.

zhongsp/TypeScript
Description: TypeScript 使用手册（中文版）翻译。http://www.typescriptlang.org

TheAlgorithms/TypeScript
Description: Algorithms and Data Structures implemented in TypeScript for beginners, following best practices.

typescript-eslint/typescript-eslint
Description: :sparkles: Monorepo for all the tooling which enables ESLint to support TypeScript

basarat/typescript-book
Description: :books: The definitive guide to TypeScript and possibly the best TypeScript book :book:. Free and Open Source 🌹



Search repositories by combining filters and printing the url of the request. This is usefull for debugging or use the url in the documentation

In [89]:
repos, url = search_public_repositories("python",created_after="2020-05-15", min_stars=1000)
print("API URL:", url)

Repositories for the topic 'python' : 

dabeaz-course/practical-python
Description: Practical Python Programming (course by @dabeaz)

openai/openai-python
Description: The official Python library for the OpenAI API

Python-World/python-mini-projects
Description: A collection of simple python mini projects to enhance your python skills

mouredev/Hello-Python
Description: Curso para aprender el lenguaje de programación Python desde cero y para principiantes. 100 clases, 44 horas en vídeo, código, proyectos y grupo de chat. Fundamentos, frontend, backend, testing, IA...

iam-veeramalla/python-for-devops
Description: Learn Python from DevOps Engineer point of you.

API URL: https://api.github.com/search/repositories?q=python+created:>=2020-05-15+stars:>=1000&per_page=10&page=1


To perform the pagination process we have to use a loop, the use case of the function is the following:

In [90]:
pagination_repos = []
for page in range(1,5):
  repositories , urls = search_public_repositories("javascript", per_page=50,page = page)
  if repositories:
    pagination_repos.extend(repositories["items"])



Repositories for the topic 'javascript' : 

airbnb/javascript
Description: JavaScript Style Guide

TheAlgorithms/JavaScript
Description: Algorithms and Data Structures implemented in JavaScript for beginners, following best practices.

trekhleb/javascript-algorithms
Description: 📝 Algorithms and data structures implemented in JavaScript with explanations and links to further readings

GitbookIO/javascript
Description: GitBook teaching programming basics with Javascript

wesbos/JavaScript30
Description: 30 Day Vanilla JS Challenge

Repositories for the topic 'javascript' : 

yml2213/javascript
Description: js自用脚本

LinkedInLearning/javascript-essential-training-2832077
Description: This repository is for the Linkedin Learning course: JavaScript Essential Training

uprock/javascript
Description: None

ironhack-labs/lab-javascript-vikings
Description: A JavaScript exercise to practice Object Oriented Programming

godotjs/javascript
Description: Javascript binding for godotengine

Repositor

# 2. Searching commits of a repository

Get the last 5 commits of a repository

In [91]:
commits_data,url = get_commits(owner="midudev",repo="aprendiendo-react",per_page=5)

The last 5 commits of midudev/aprendiendo-react
2025-01-03T11:27:31Z- Miguel Ángel Durán
Merge pull request #109 from GianninaGit/patch-3

Avoid translate of "Tic tac toe" in spanish App.jsx

2025-01-03T11:27:16Z- Miguel Ángel Durán
Merge pull request #108 from GianninaGit/patch-2

Remove border in .winner .win from index.css

2025-01-03T11:26:22Z- Miguel Ángel Durán
Merge pull request #107 from leo-dillon/master

Error en la carga de imagenes de Creación de un ecommerce con carrito de compras

2024-05-24T13:33:33Z- Miguel Ángel Durán
Update README.md

2024-03-10T00:36:27Z- Giannina
Avoid translate of "Tic tac toe" in spanish App.jsx

A pesar de tener en Layout.astro <html lang="es">, cuando el browser está en español, se traduce el título "Tic tac toe". He añadido translate="no" en App.jsx para evitar que esto ocurra :)



Filtering commits from a specific date

In [92]:
commits_data,url = get_commits(owner="midudev",repo="la-velada-web-oficial",per_page=5,since="2024-12-01T00:00:00Z")

The last 5 commits of midudev/la-velada-web-oficial
2025-05-21T14:40:00Z- midudev
Remove experimental flag

2025-05-21T14:35:19Z- midudev
Upgrade dependencies

2025-05-21T14:33:29Z- midudev
Merge branch 'main' of github.com:midudev/la-velada-web-oficial

2025-05-21T14:33:27Z- midudev
fix: ajustar formato de código y agregar metadatos estructurados para el evento

2025-05-20T08:46:56Z- Miguel Ángel Durán
Merge pull request #1393 from BlondyMartinez/main

fix: missing hyperlink to infojobs



Filtering 10 commits within a date range

In [93]:
commits_data,url = get_commits(
    owner="midudev",
    repo="javascript-100-proyectos",
    since="2024-03-01T00:00:00Z",
    until="2024-04-01T00:00:00Z",
    per_page=10
)

The last 10 commits of midudev/javascript-100-proyectos
2024-03-31T23:29:52Z- Miguel Ángel Durán
Update README.md

2024-03-31T23:29:21Z- Miguel Ángel Durán
add new project

2024-03-28T21:21:36Z- Miguel Ángel Durán
Merge branch 'main' of github.com:midudev/javascript-100-proyectos

2024-03-28T21:21:16Z- Miguel Ángel Durán
Upgrade dependencies

2024-03-28T21:19:56Z- Miguel Ángel Durán
Update README.md

2024-03-28T21:19:06Z- Miguel Ángel Durán
Add new project

2024-03-28T18:45:51Z- Miguel Ángel Durán
Fix initial time

2024-03-28T18:43:45Z- Miguel Ángel Durán
Add data and project 03

2024-03-05T08:39:59Z- Miguel Ángel Durán
Merge pull request #26 from JaimeGamm/main

cambios en dimension de las imagenes y una imagen no tenia el atribut…

2024-03-05T08:38:59Z- Miguel Ángel Durán
Merge pull request #27 from EduWTR/title-to-iframe

fix: add title to iframe for better accessibility ( web )



To perform the pagination process we have to use a loop, the use case of the function is the following:

In [94]:
results_per_page = 10
pages = 3
for page in range(1, pages + 1):
    print(f"\nPage: {page}")
    commits, url = get_commits(
        owner="midudev",
        repo="javascript-100-proyectos",
        per_page=results_per_page,
        page=page
    )
    if commits:
      print(f" Fetched {len(commits)} commits from {url}")
    else:
      print("No more commits found")


Page: 1
The last 10 commits of midudev/javascript-100-proyectos
2024-12-23T09:43:13Z- midudev
Add project 12: Animación de fondo por scroll with GitHub and Vercel links in README.md

2024-12-20T14:17:07Z- Miguel Ángel Durán
Merge branch 'main' of github.com:midudev/javascript-100-proyectos

2024-12-20T14:16:59Z- Miguel Ángel Durán
fix lo

2024-12-20T08:29:34Z- midudev
Add project 12: Animación por Scroll with YouTube link

2024-12-20T08:29:14Z- midudev
Add project 12

2024-12-20T08:28:54Z- midudev
Refactor: Center content and change div to aside in project 11

2024-12-20T08:28:42Z- midudev
Add image project 12

2024-12-20T08:23:12Z- Miguel Ángel Durán
Merge pull request #72 from GFrancV/main

Fix: Add YouTube link in project  11

2024-12-19T12:09:17Z- GFrancV
Fix: Add YouTube link in project  11

2024-12-19T11:30:02Z- Miguel Ángel Durán
Add images

 Fetched 10 commits from https://api.github.com/repos/midudev/javascript-100-proyectos/commits

Page: 2
The last 10 commits of midudev/jav

## 3. Examples for getting repo content

Show all the root files of the repository

In [104]:
file_links = repo_content(owner="midudev",repo="porfolio.dev")

Owner:  midudev 
 Name of the repo: porfolio.dev
 Content: [{'name': '.gitignore', 'path': '.gitignore', 'sha': 'b871ec039a2c06b90459add2d6385c5a0107c8cd', 'size': 268, 'url': 'https://api.github.com/repos/midudev/porfolio.dev/contents/.gitignore?ref=main', 'html_url': 'https://github.com/midudev/porfolio.dev/blob/main/.gitignore', 'git_url': 'https://api.github.com/repos/midudev/porfolio.dev/git/blobs/b871ec039a2c06b90459add2d6385c5a0107c8cd', 'download_url': 'https://raw.githubusercontent.com/midudev/porfolio.dev/main/.gitignore', 'type': 'file', '_links': {'self': 'https://api.github.com/repos/midudev/porfolio.dev/contents/.gitignore?ref=main', 'git': 'https://api.github.com/repos/midudev/porfolio.dev/git/blobs/b871ec039a2c06b90459add2d6385c5a0107c8cd', 'html': 'https://github.com/midudev/porfolio.dev/blob/main/.gitignore'}}, {'name': '.vscode', 'path': '.vscode', 'sha': '0c33c7c3c06928c95be8c68f9f15a508c844a3dc', 'size': 0, 'url': 'https://api.github.com/repos/midudev/porfolio.dev/

Get the content of a subdirectory

In [105]:
file_links = repo_content(owner="midudev",repo="porfolio.dev",path="public")

Owner:  midudev 
 Name of the repo: porfolio.dev
 Content: [{'name': 'favicon.svg', 'path': 'public/favicon.svg', 'sha': '382ee01227e1de62a8eb4c6c433505cf54a8debb', 'size': 514, 'url': 'https://api.github.com/repos/midudev/porfolio.dev/contents/public/favicon.svg?ref=main', 'html_url': 'https://github.com/midudev/porfolio.dev/blob/main/public/favicon.svg', 'git_url': 'https://api.github.com/repos/midudev/porfolio.dev/git/blobs/382ee01227e1de62a8eb4c6c433505cf54a8debb', 'download_url': 'https://raw.githubusercontent.com/midudev/porfolio.dev/main/public/favicon.svg', 'type': 'file', '_links': {'self': 'https://api.github.com/repos/midudev/porfolio.dev/contents/public/favicon.svg?ref=main', 'git': 'https://api.github.com/repos/midudev/porfolio.dev/git/blobs/382ee01227e1de62a8eb4c6c433505cf54a8debb', 'html': 'https://github.com/midudev/porfolio.dev/blob/main/public/favicon.svg'}}, {'name': 'me.png', 'path': 'public/me.png', 'sha': '6ca44a4956af206843439c5f395d793f8ce4e62b', 'size': 81008, 

Requesting a specific file

In [106]:
file_links = repo_content(owner="midudev",repo="porfolio.dev",path="public/favicon.svg")

Owner:  midudev 
 Name of the repo: porfolio.dev
 Content: {'name': 'favicon.svg', 'path': 'public/favicon.svg', 'sha': '382ee01227e1de62a8eb4c6c433505cf54a8debb', 'size': 514, 'url': 'https://api.github.com/repos/midudev/porfolio.dev/contents/public/favicon.svg?ref=main', 'html_url': 'https://github.com/midudev/porfolio.dev/blob/main/public/favicon.svg', 'git_url': 'https://api.github.com/repos/midudev/porfolio.dev/git/blobs/382ee01227e1de62a8eb4c6c433505cf54a8debb', 'download_url': 'https://raw.githubusercontent.com/midudev/porfolio.dev/main/public/favicon.svg', 'type': 'file', 'content': 'PHN2ZwogIHhtbG5zPSJodHRwOi8vd3d3LnczLm9yZy8yMDAwL3N2ZyIKICB3\naWR0aD0iMjQiCiAgaGVpZ2h0PSIyNCIKICB2aWV3Qm94PSIwIDAgMjQgMjQi\nCiAgc3Ryb2tlLXdpZHRoPSIyIgogIHN0cm9rZT0iY3VycmVudENvbG9yIgog\nIGZpbGw9Im5vbmUiCiAgc3Ryb2tlLWxpbmVjYXA9InJvdW5kIgogIHN0cm9r\nZS1saW5lam9pbj0icm91bmQiCiAgPjxwYXRoIHN0cm9rZT0ibm9uZSIgZD0i\nTTAgMGgyNHYyNEgweiIgZmlsbD0ibm9uZSI+PC9wYXRoPjxwYXRoCiAgICBk\nPSJNNyA4bC00IDRsNCA0Ij48L3Bhd

# Troubleshooting Guide

Checking rate limit

In [98]:
rate_limit(headers)

Remaining requests: 4975
Reset datetime: 2025-06-26 22:49:52


Checking Auth Token. If your token is verified the function displays your username

In [99]:
test_connection()

Succesful opperation. Authenticated user: manu97a


There is a possibility that when using the API the limit allowed by GitHub may be exceeded, that is the reason why error handling has been added to the functions.
The limits per page cannot be greater than 100 and the product between per_page and page cannot be greater than 1000

In [100]:
get_commits(owner="midudev",repo="aprendiendo-react",per_page=200,page=25)

Error. per_page cannot be greater than 100 (Github Limit)


(None, None)

In [101]:
get_commits(owner="midudev",repo="aprendiendo-react",per_page=50,page=25)

Error. per_page*page cannot be greater than 1000 results (Github Limit)


(None, None)

In [102]:
repos, url = search_public_repositories("javascript",per_page=150)

Error. per_page cannot be greater than 100 (Github Limit)


In [103]:
repos, url = search_public_repositories("javascript",per_page=50,page=25)

Error. per_page*page cannot be greater than 1000 results (Github Limit)
